In [1]:
import pandas as pd
titles = [l.strip() for l in open("hackernews").readlines()]
df = pd.DataFrame(titles).drop_duplicates()
df = pd.DataFrame(df[0].tolist(), columns=["title"])
df

# df = pd.read_csv("hacker_news_sample.csv")["title"].dropna()
# df = pd.DataFrame(df, columns=["title"])
# df

,title
0,The first Oxide rack being prepared for custom...
1,"Build your own Docker with Linux namespaces, c..."
2,Make your programs run faster by better using ...
3,An open source web-based flashcard studying sy...
4,Millions of GitHub repos likely vulnerable to ...
...,...
1820,U.S. Feds Seized Nearly $1B in Bitcoin from Wa...
1821,Facebook was used as a proxy by web scraping bots
1822,How is Visual Basic still ranked #6 programmin...
1823,Confessions of a voter fraud: I was a master a...


In [2]:
import re
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

def preprocess(text:str):
    # remove the url at the end
    text = text[:text.find('(')]
    # split text
    text = text.lower()
    words = text.split()
    ret = []
    for word in words:
        # strip whitespaces
        word = word.strip()
        # remove stopwords
        if word in stopwords_en:
            continue
        # regex
        word = re.sub(r'[0-9]+', '', word)
        word = re.sub(r'[^a-z]+', '', word)
        word = re.sub(r'\W', '', word)
        # remove words len<3
        if len(word) < 3:
            continue
        ret.append(word)
    return ' '.join(ret)

df["text"] = df["title"].apply(preprocess)
df

,title,text
0,The first Oxide rack being prepared for custom...,first oxide rack prepared customer shipment
1,"Build your own Docker with Linux namespaces, c...",build docker linux namespaces cgroups chroot
2,Make your programs run faster by better using ...,make programs run faster better using data cache
3,An open source web-based flashcard studying sy...,open source webbased flashcard studying system
4,Millions of GitHub repos likely vulnerable to ...,millions github repos likely vulnerable repoja...
...,...,...
1820,U.S. Feds Seized Nearly $1B in Bitcoin from Wa...,feds seized nearly bitcoin wallet linked silk roa
1821,Facebook was used as a proxy by web scraping bots,facebook used proxy web scraping bot
1822,How is Visual Basic still ranked #6 programmin...,visual basic still ranked programming language
1823,Confessions of a voter fraud: I was a master a...,confessions voter fraud master fixing mailin b...


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(max_features=10000)
vectorized = vectorizer.fit_transform(df["text"].tolist())
# vectorized.resize((len(df["text"]),300))
print(vectorized.shape)

X = [
    # "Excellence is a habit, but so is failure (awesomekling.github.io)",
    "Meta releases Intermediate Graphics Library (khronos.org)",
]
X = [preprocess(x) for x in X]
print(X)

test_vec = vectorizer.transform(X)
# test_vec.resize((len(X),300))
print(test_vec, test_vec.shape)

(1825, 5017)
['meta releases intermediate graphics library']
  (0, 3703)	0.4971177293335161
  (0, 2789)	0.5243146703243883
  (0, 2541)	0.4506242516194009
  (0, 1962)	0.5243146703243883 (1, 5017)


In [16]:
cos_sim = cosine_similarity(vectorized,test_vec)
for i in range(test_vec.shape[0]):
    df["similarity"] = [x[0] for x in cos_sim]

top_k = (
    df.sort_values(by="similarity", ascending=False)
        .head(10)
        .query("similarity > 0")
    )
print(X)
top_k

['meta releases intermediate graphics library']


,title,text,similarity
73,Meta announces its Quest 3 VR headset (theverg...,meta announces quest headset,0.271165
897,Intel Xe Graphics: An Interview with VP Lisa P...,intel graphics interview lisa pearc,0.250634
927,Building the Mathematical Library of the Future,building mathematical library futur,0.218391
186,The C Standard Library Function Isspace() Depe...,standard library function isspace,0.211648
780,"AWS releases Glue Databrew, a visual ETL tool",aws releases glue databrew visual etl,0.205350
274,JobRunr: A library for background processing i...,jobrunr library background processing java,0.184280
180,Nvidia releases new AI chip with 480GB CPU RAM...,nvidia releases new chip cpu ram gpu ram,0.166197
521,Show HN: React library for showing speedometer...,show react library showing speedometer like ga...,0.161847
694,Fast UI Draw is a library that provides a high...,fast draw library provides higher performance ...,0.147349


## LDA

In [31]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(
    n_components=10,
    random_state=0,
)
Xvec = lda.fit_transform(vectorized)
print(Xvec)

[[0.02910137 0.02910466 0.02910355 ... 0.73807824 0.02910262 0.02910423]
 [0.02919538 0.02919274 0.02921517 ... 0.73713804 0.02920481 0.02920114]
 [0.02626277 0.02626175 0.0262685  ... 0.76360792 0.02626687 0.02626391]
 ...
 [0.02912581 0.02912386 0.02912304 ... 0.73787064 0.02912285 0.02912813]
 [0.0274826  0.02748652 0.02748301 ... 0.02748188 0.02748303 0.02748174]
 [0.02748726 0.02748749 0.02748853 ... 0.75261336 0.02748731 0.02748685]]


In [32]:
import matplotlib.pyplot as plt
pltX = [dot[0] for dot in Xvec]
pltY = [dot[1] for dot in Xvec]

newX,newY = lda.transform(test_vec)[0]
print(newX, newY)

plt.scatter(pltX,pltY)
plt.scatter(newX,newY,c="red")

ValueError: too many values to unpack (expected 2)